In [ ]:
#setup credentials for BigQuery
#note that the first 1TB of querying is free, but then a fee applies after that
from google.cloud import bigquery
credentials_path = './test-bigquery-ot-dcacff312913.json'
client = bigquery.Client.from_service_account_json(credentials_path)

In [ ]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
query='''
WITH ranked_genes AS (
    SELECT locus2gene.study_id, 
        locus2gene.chrom, locus2gene.pos, locus2gene.ref, locus2gene.alt, 
        study_metadata.*, 
        genes.gene_name, locus2gene.y_proba_full_model
    
    FROM `bigquery-public-data.open_targets_genetics.locus2gene` AS locus2gene
    
    #get GWAS metadata
    INNER JOIN `bigquery-public-data.open_targets_genetics.studies` AS study_metadata
    ON locus2gene.study_id = study_metadata.study_id
    
    #get HGNC IDs
    INNER JOIN `bigquery-public-data.open_targets_genetics.genes` AS genes
    ON locus2gene.gene_id = genes.gene_id

    
    #filter on the Neale Lab UKBB analysis
    WHERE locus2gene.study_id LIKE '%NEALE2%'
        
        #optional: filter on a disease
        AND LOWER(study_metadata.trait_reported) LIKE '%coeliac%'
        
        #remove the "raw" neale lab results -- I'm not sure what this is
        AND locus2gene.study_id NOT LIKE '%raw%'

        #filter to a l2g score threshold
        AND locus2gene.y_proba_full_model > 0.2

    )
SELECT * from ranked_genes
'''

# Run the query
query_job = client.query(query)

l2g = query_job.to_dataframe()
l2g